In [2]:
import os
import requests
from urllib.parse import urljoin
from datetime import datetime
import sys

class PrintToFile:
    def __init__(self, file_name=None):
        if file_name is None:
            file_name = self.get_notebook_name() + ".txt"
        self.file_name = file_name

    def get_notebook_name(self):
       
        return "encoderDecoder_MLP_redNeuronalProfunda"

    def print(self, *args, **kwargs):
        fecha_hora = datetime.now().strftime("[%d/%m/%Y %H:%M:%S]")
        mensaje = fecha_hora + " " + " ".join(map(str, args))

        with open(self.file_name, "a") as f:
            original_stdout = sys.stdout
            sys.stdout = f

            try:
                print(mensaje, **kwargs)
            finally:
                sys.stdout = original_stdout

    def reprint(self, *args, **kwargs):
        self.print(*args, **kwargs)
        print(*args, **kwargs)

# Crear una instancia de la clase PrintToFile
p2f = PrintToFile()

# Imprimir en el archivo usando la función print de la clase
p2f.print("Este es un mensaje.")
p2f.print("Otro mensaje más.")

# Continuar imprimiendo normalmente en la consola
print("Este mensaje se verá en la consola.")


Este mensaje se verá en la consola.


Explicación de Cada Modelo

    MLPClassifier: Un modelo de red neuronal que puede capturar relaciones no lineales en los datos. Es adecuado para problemas con datos tabulares.

    XGBoost: Un modelo de boosting basado en árboles que maneja bien datos tabulares y puede capturar interacciones complejas entre características.

    CatBoost: Un modelo de boosting que maneja de manera eficiente características categóricas y puede ser más rápido de entrenar en comparación con XGBoost.

Consideraciones Adicionales

    Preprocesamiento de Datos: Asegúrate de que el preprocesamiento (como la codificación de características categóricas) sea consistente para todos los modelos.

    Ajuste de Hiperparámetros: Puedes ajustar los hiperparámetros de los modelos para mejorar el rendimiento, utilizando técnicas como la búsqueda en cuadrícula o la optimización bayesiana.

    Validación Cruzada: Utiliza validación cruzada para evaluar el rendimiento del modelo y evitar el sobreajuste.

In [3]:
from datetime import datetime

class TimeTracker:
    def __init__(self, start_time = None):
        """
        Inicializa el objeto TimeTracker con el tiempo de inicio actual.
        """
        if start_time == None:
            self.start_time = datetime.now()
        else:
            self.start_time = start_time 
    
    def dime_que_llevo(self, current_time = None):
        """
        Calcula la diferencia de tiempo desde el inicio y la devuelve formateada en horas, minutos y segundos.
        
        Returns:
            str: La diferencia de tiempo formateada como 'HH:MM:SS'.
        """
        # Obtiene el tiempo actual
        if (current_time == None):
            current_time = datetime.now()
        
        # Calcula la diferencia de tiempo entre el tiempo actual y el tiempo de inicio
        time_difference = current_time - self.start_time

        # Extrae los segundos totales de la diferencia
        total_seconds = int(time_difference.total_seconds())

        # Calcula horas, minutos y segundos
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60

        # Formatea la diferencia como HH:MM:SS
        formatted_difference = f"{seconds:02} seg.s"
        if (minutes > 0):
            formatted_difference = f"{minutes:02} min.s y {formatted_difference}"
        if (hours > 0):
            formatted_difference = f"{hours:02} horas {formatted_difference}" 
            
        self.start_time = datetime.now()
        
        return formatted_difference

# Ejemplo de uso
tracker = TimeTracker()
# Simulación de esperar algún tiempo
import time
time.sleep(5)  # Espera 5 segundos para demostrar el cálculo
p2f.reprint(tracker.dime_que_llevo())  # Salida: '00:00:05'



05 seg.s


In [7]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GroupShuffleSplit
from tqdm import tqdm
import joblib
import os
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


# Paso 0: Cargar el fichero CSV y el JSON con los literales de categorías
#df = pd.read_csv('./item_features_semirelleno_con_literales_actualizado.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_2.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_3.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_4.csv')
# El 5 fue asociando por la longitud de los valores únicos
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_6.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_7.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_8.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_9.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_10.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_11.csv')
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_12.csv')
# El 13 fue asociando por la longitud de los valores únicos
#df = pd.read_csv('./item_features_rellenado_con_literales_MLP_13.csv')
df = pd.read_csv('./item_features_rellenado_con_literales_MLP_14_con_values_id.csv')

with open('./literales_ultimate.json', 'r', encoding='utf-8') as f:
    category_data = json.load(f)

# Contar los valores distintos de feature_value_id por cada feature_category_id
category_value_counts = df.groupby('feature_category_id')['feature_value_id'].nunique()

# Filtrar las categorías en category_data para que coincidan con los conteos de feature_value_id
filtered_category_data = {}
for category_literal, values in category_data.items():
    for feature_category_id, count in category_value_counts.items():
        if len(values) == count:
            filtered_category_data[category_literal] = values

# Convertir las claves de filtered_category_data a una lista
category_literals = list(filtered_category_data.keys())

# Identificar categorías válidas que no están en feature_category_literal del DataFrame
df_category_literals = df['feature_category_literal'].unique()
missing_categories = set(category_literals) - set(df_category_literals)

# Codificar todos los literales posibles desde el fichero JSON
encoder_category_literal = LabelEncoder()
encoder_category_literal.fit(category_literals)

# Separar el DataFrame en dos: uno con valores completos y otro con valores faltantes
df['filled_literal'] = df['feature_category_literal'].isin(category_literals)


# Para los datos completos
#df_complete['feature_category_literal_encoded'] = encoder_category_literal.transform(df_complete['feature_category_literal'])
def safe_transform(row):
    """
    Transforma la categoría si 'filled_literal' es True.
    
    Args:
        row (pd.Series): Fila del DataFrame que contiene las columnas 'feature_category_literal' y 'filled_literal'.
        
    Returns:
        Transformed value if 'filled_literal' is True, else None.
    """
    if row['filled_literal']:  # Verifica el valor de 'filled_literal'
        try:
            return encoder_category_literal.transform([row['feature_category_literal']])[0]
        except ValueError:
            return None  # O un valor que consideres adecuado si la categoría no se encuentra en el encoder
    else:
        return None  # O un valor que consideres adecuado para entradas vacías

# Aplicar la función safe_transform a cada fila en el DataFrame
#df_complete['feature_category_literal_encoded'] = df_complete.apply(safe_transform, axis=1)
df['feature_category_literal_encoded'] = df.apply(safe_transform, axis=1)

df_complete = df[df['filled_literal']]
df_missing = df[~df['filled_literal']]

# Opcional: Eliminar la columna auxiliar si ya no se necesita
df = df.drop(columns=['filled_literal'])
df_missing = df_missing.drop(columns=['filled_literal'])
df_complete = df_complete.drop(columns=['filled_literal'])

# Definir X y y para el modelo
X = pd.get_dummies(df_complete[['feature_category_id']])
y = df_complete['feature_category_literal_encoded']

X_train = X
y_train = y

p2f.reprint("Datos preparados para el modelo MLPClassifier...")

Datos preparados para el modelo MLPClassifier...


In [5]:
import numpy as np
import time
#from catboost import CatBoostClassifier, Pool
### Diccionario para almacenar los resultados
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
results = {}

# Cargar o entrenar el modelo MLPClassifier
model_path = "mlp_model.pkl"
existe_modelo = os.path.exists(model_path)
existe_modelo = False
if existe_modelo:
    p2f.reprint("Cargando el modelo desde el disco...")
    mlp = joblib.load(model_path)
else:
    p2f.reprint("Entrenando el modelo MLPClassifier...")
    
    tunear = False
    if tunear:
        groups = df_complete['item_id']

        # Usar GroupShuffleSplit para mantener los item_id juntos
        gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
        train_idx, test_idx = next(gss.split(X, y, groups=groups))

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        #train_idx, val_idx = next(gss.split(X_train, y, groups=groups))
        #X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        #y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]    

        # Definir los parámetros para GridSearch, incluyendo estrategias de validación cruzada
        param_grid = {
            'mlp__hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
            'mlp__activation': ['relu', 'tanh'],
            'mlp__solver': ['adam', 'sgd']
            #'cv_strategy': [KFold(n_splits=5, shuffle=True, random_state=42),
            #                StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            #                TimeSeriesSplit(n_splits=5)]
        }

        # Definir el modelo base
        mlp = MLPClassifier(
            max_iter=500,  # Número máximo de iteraciones
            verbose=True,  # Imprimir información del entrenamiento
            early_stopping=True,  # Activar early stopping
            validation_fraction=0.1,  # Fracción de datos para validación
            n_iter_no_change=10,  # Número de iteraciones sin mejora antes de detenerse
            random_state=42  # Semilla para la aleatoriedad
        )
        
        # Crear un pipeline que incluya escalado y el modelo
        pipeline = Pipeline([
            ('scaler', StandardScaler()),  # Escalado de características
            ('mlp', mlp)                   # Modelo MLP
        ])
        
        # Definir los parámetros para GridSearch, incluyendo estrategias de validación cruzada
        param_grid = {
            'mlp__hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
            'mlp__activation': ['relu', 'tanh'],
            'mlp__solver': ['adam', 'sgd']
        }

        # Definir diferentes estrategias de validación cruzada
        cv_strategies = {
            'KFold': KFold(n_splits=5, shuffle=True, random_state=42),
            'StratifiedKFold': StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            'TimeSeriesSplit': TimeSeriesSplit(n_splits=5)
        }

        # Configurar GridSearchCV con el pipeline
        tracker = TimeTracker()
        # Ejecutar GridSearchCV para cada estrategia de validación cruzada
        for cv_name, cv_strategy in cv_strategies.items():
            p2f.reprint("\n---------------------------------------------------------------")
            p2f.reprint(f"\nEvaluando con {cv_name}")
            
            #p2f.reprint("LLevo ",tracker.dime_que_llevo())
            
            grid_search = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grid,
                cv=cv_strategy,  # Estrategia de validación cruzada
                n_jobs=-1,
                verbose=2
                #scoring=None,  # Puedes definir una métrica de scoring si no quieres usar la predeterminada
            )
            
            #p2f.reprint("(2)LLevo ",tracker.dime_que_llevo())
            
            # Ajustar GridSearchCV
            grid_search.fit(X_train, y_train)
            
            p2f.reprint("LLevo ",tracker.dime_que_llevo())
            
            # Mostrar los mejores parámetros
            p2f.reprint(f'Best parameters: {grid_search.best_params_}')
            
            p2f.reprint('Testeando!')
            
            # Evaluar el mejor modelo en el conjunto de prueba
            best_mlp = grid_search.best_estimator_
            y_test_pred = best_mlp.predict(X_test)
            
            # Calcular métricas de rendimiento
            accuracy = accuracy_score(y_test, y_test_pred)
            f1 = f1_score(y_test, y_test_pred, average='weighted')
            roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1]) if len(set(y_test)) == 2 else None  # Para problemas binarios

            #p2f.print(f'Accuracy en el conjunto de prueba: {accuracy_score(y_test, y_test_pred)}')
            #p2f.print(y_test_pred)

            # Almacenar los resultados
            results[cv_name] = {
                'best_params': grid_search.best_params_,
                'best_mlp': best_mlp,
                'accuracy': accuracy,
                'f1_score': f1,
                'roc_auc': roc_auc
            } 
            # Mostrar los resultados
            p2f.reprint(f'Best parameters with {cv_name}: {grid_search.best_params_}')
            p2f.reprint(f'Accuracy on test set with {cv_name}: {accuracy:.4f}')
            p2f.reprint(f'F1 Score on test set with {cv_name}: {f1:.4f}')
            if roc_auc is not None:
                p2f.reprint(f'ROC AUC on test set with {cv_name}: {roc_auc:.4f}')        
        
        # Comparar los resultados de todas las estrategias
        p2f.reprint("\n---------------------------------------------------------------")
        p2f.reprint("\nComparación de estrategias de validación cruzada:")
        for cv_name, result in results.items():
            p2f.print(f"{cv_name}:")
            p2f.print(f"  Best Params: {result['best_params']}")
            #p2f.print(f"  Accuracy: {result['accuracy']:.4f}")
            #p2f.print(f"  F1 Score: {result['f1_score']:.4f}")
            #if result['roc_auc'] is not None:
            #    p2f.print(f"  ROC AUC: {result['roc_auc']:.4f}")
            p2f.print()  

        #mlp = results[0].best_mlp
        mlp = results['KFold']['best_mlp']

        p2f.reprint("Tuneando, no salvo a fichero el modelo.")
    else:
       
        tracker = TimeTracker()
        
        best_parameters = {'mlp__activation': 'relu', 'mlp__hidden_layer_sizes': (50,), 'mlp__solver': 'adam'}
        
        mlp = MLPClassifier(
            activation=best_parameters['mlp__activation'],  # 'relu'
            hidden_layer_sizes=best_parameters['mlp__hidden_layer_sizes'],  # (50,)
            solver=best_parameters['mlp__solver'],  # 'adam'
            max_iter=500,  # Número máximo de iteraciones
            verbose=True,  # Imprimir información del entrenamiento
            early_stopping=True,  # Activar early stopping
            validation_fraction=0.1,  # Fracción de datos para validación
            n_iter_no_change=10,  # Número de iteraciones sin mejora antes de detenerse
            random_state=42  # Semilla para la aleatoriedad
        )

        # Entrenar el modelo con los datos de entrenamiento
        mlp.fit(X_train, y_train)
        
        p2f.reprint("(best_model) Tardo en entrenar el modelo ",tracker.dime_que_llevo())
        
        
        joblib.dump(mlp, model_path)
        p2f.reprint(f"Modelo guardado en {model_path}.")  

Entrenando el modelo MLPClassifier...
Iteration 1, loss = 2.45892292
Validation score: 0.493677
Iteration 2, loss = 1.67929428
Validation score: 0.604693
Iteration 3, loss = 1.39610196
Validation score: 0.546356
Iteration 4, loss = 1.22202235
Validation score: 0.665454
Iteration 5, loss = 1.10585095
Validation score: 0.688846
Iteration 6, loss = 1.02009941
Validation score: 0.737246
Iteration 7, loss = 0.95077019
Validation score: 0.813460
Iteration 8, loss = 0.89117732
Validation score: 0.813460
Iteration 9, loss = 0.83868405
Validation score: 0.771003
Iteration 10, loss = 0.79234411
Validation score: 0.765155
Iteration 11, loss = 0.75000540
Validation score: 0.792992
Iteration 12, loss = 0.70973588
Validation score: 0.844292
Iteration 13, loss = 0.67332991
Validation score: 0.796724
Iteration 14, loss = 0.64120860
Validation score: 0.844292
Iteration 15, loss = 0.61198053
Validation score: 0.844292
Iteration 16, loss = 0.58377126
Validation score: 0.854633
Iteration 17, loss = 0.5589

In [6]:
# Crear un diccionario para almacenar las asociaciones feature_category_id -> feature_category_literal
category_id_to_literal = {}

# Paso 2: Imputar los valores faltantes de 'feature_category_literal'
p2f.print("Asignando 'feature_category_literal'...")

if df_missing.empty:
    p2f.reprint("No hay valores faltantes en 'feature_category_literal'. Nada que imputar.")
else:
    p2f.reprint("Procediendo a imputar valores faltantes en 'feature_category_literal'...")
    num_asociaciones = 0
    failed_feature_category_id = []
    group_count = 0  # Contador de grupos
    elementos_asociados = 0
    for item_id, group in tqdm(df_missing.groupby('item_id')):
        
        group_count += 1  # Incrementar el contador
        if group_count % 5000 == 0:
            p2f.reprint(f"({group_count})::LLevo ",tracker.dime_que_llevo())
            
        p2f.print(f"\nProcesando grupo {group_count} para item_id {item_id}")
        #print(f"Contenido del grupo:\n{group}")

        if group['feature_category_literal'].isna().any():

            X_missing = pd.get_dummies(group[['feature_category_id']])

            missing_cols = set(X_train.columns) - set(X_missing.columns)
            for col in missing_cols:
                X_missing[col] = 0
            X_missing = X_missing[X_train.columns]

            for i, (index, row) in enumerate(group[group['feature_category_literal'].isna()].iterrows()):
                feature_category_id = row['feature_category_id']

                # Si la categoría ya se asignó previamente, usar esa asociación
                if feature_category_id in category_id_to_literal:
                    elementos_asociados += 1
                    assigned_category = category_id_to_literal[feature_category_id]
                    df.at[index, 'feature_category_literal'] = assigned_category
                    p2f.print(f"Reutilizado '{assigned_category}' de asociaciones anteriores para 'feature_category_id' {feature_category_id} en el índice {index} para item_id {item_id}.")
                else:
                    # Predecir solo si no hay una asociación previa
                    predicted_categories = mlp.predict_proba(X_missing)
                    top_indices = predicted_categories.argsort(axis=1)[:, -20:][:, ::-1]  # Obtener los 20 candidatos más probables
                    decoded_candidates = [[encoder_category_literal.inverse_transform([idx])[0] for idx in indices] for indices in top_indices]

                    # Evaluar candidatos posibles con las nuevas condiciones
                    possible_candidates = []
                    for cat in decoded_candidates[i]:
                        cat_length = len(filtered_category_data.get(cat, []))
                        num_possible_values = category_value_counts.get(feature_category_id, 0)
                        is_in_missing_categories = cat in missing_categories

                        if cat_length == num_possible_values and is_in_missing_categories:
                            possible_candidates.append(cat)

                    if possible_candidates:
                        p2f.print(f"Posibles candidatos para {feature_category_id}: {possible_candidates}")
                        assigned_category = possible_candidates[0]
                        df.at[index, 'feature_category_literal'] = assigned_category
                        category_id_to_literal[feature_category_id] = assigned_category  # Guardar la asociación
                        p2f.print(f"Asignado '{assigned_category}' a 'feature_category_literal' en el índice {feature_category_id} para item_id {item_id}.")
                        num_asociaciones += 1
                        elementos_asociados += 1
                        # Eliminar la categoría asignada de missing_categories
                        if assigned_category in missing_categories:
                            missing_categories.remove(assigned_category)
                            p2f.print(f"Eliminado '{assigned_category}' de missing_categories.")
                    else:
                        # Añadir solo si no está ya en la lista
                        if feature_category_id not in failed_feature_category_id:  
                            failed_feature_category_id.append(feature_category_id)
                            
    p2f.reprint(f"({group_count})::LLevo ",tracker.dime_que_llevo())   
    p2f.reprint(f"\nElementos actualizados {elementos_asociados}") 
    p2f.reprint(f"\nNúmero total de grupos procesados: {group_count}, nº de asociaciones realizadas {num_asociaciones}")
    p2f.reprint(f"\nAsociaciones creadas ({len(category_id_to_literal)}:\n{category_id_to_literal}")
    p2f.reprint(f"\nCategorías no asociadas ({len(failed_feature_category_id)}):\n{failed_feature_category_id}")    
    
    # Guardar el resultado, para iterar de nuevo, en vez de item_features_semirelleno_con_literales_actualizado, 
    # usar este otro, cambiar este nombre y asi sucesivamente,
    #df.to_csv('./item_features_rellenado_con_literales_MLP.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_2.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_3.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_4.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_5.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_7.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_8.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_9.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_10.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_11.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_12.csv', index=False)
    #df.to_csv('./item_features_rellenado_con_literales_MLP_13.csv', index=False)
    df.to_csv('./item_features_rellenado_con_literales_MLP_14.csv', index=False)
    
    p2f.reprint("Archivo guardado con éxito.")

Procediendo a imputar valores faltantes en 'feature_category_literal'...


 24%|██████████████████▋                                                          | 5004/20562 [01:40<06:09, 42.08it/s]

(5000)::LLevo  01 min.s y 48 seg.s


 49%|████████████████████████████████████▉                                       | 10006/20562 [03:20<03:33, 49.45it/s]

(10000)::LLevo  01 min.s y 39 seg.s


...

(15000)::LLevo  01 min.s y 39 seg.s


 97%|█████████████████████████████████████████████████████████████████████████▉  | 20005/20562 [06:39<00:12, 42.95it/s]

(20000)::LLevo  01 min.s y 39 seg.s


100%|████████████████████████████████████████████████████████████████████████████| 20562/20562 [06:50<00:00, 50.04it/s]


(20562)::LLevo  11 seg.s

Elementos actualizados 0

Número total de grupos procesados: 20562, nº de asociaciones realizadas 0

Asociaciones creadas (0:
{}

Categorías no asociadas (16):
[59, 65, 41, 34, 57, 49, 38, 37, 36, 67, 6, 52, 43, 10, 71, 40]
Archivo guardado con éxito.


In [8]:
#df_complete = df_complete.drop(columns=['item_id_2'])
df_complete = df_complete.drop(columns=['feature_category_literal_encoded'])


# Me quedo sólo con lo que he asignado de literales, mi encoder/decoder, por si ya no hiciera nada más, no asignará más...
yyyyMMdd_HHmmss = datetime.now().strftime("%Y%m%d_%H%M%S")
df_complete.to_csv(f"./item_features_encoderDecoder_{yyyyMMdd_HHmmss}.csv", index=False)
p2f.reprint("Archivo original de base, sólo con los rellenos, guardado con éxito.")

# Paso 3: Validación y Testing
#p2f.reprint("Validando el modelo con el solo con los rellenosconjunto de prueba...")
#y_pred = mlp.predict(X_test)
#p2f.print("Reporte de clasificación en el conjunto de prueba:")

#print(encoder_category_literal.classes_)
#print("Clases en y_test:", set(y_test))
#print("Clases en y_pred:", set(y_pred.flatten().tolist()))
#print (y_pred)
#p2f.print(classification_report(y_test, y_pred, target_names=encoder_category_literal.classes_))

# Guardar el resultado del modelo en el conjunto de prueba
#test_results = pd.DataFrame({
#    'item_id': filtered_df.iloc[test_idx]['item_id'],
#    'feature_category_id': filtered_df.iloc[test_idx]['feature_category_id'],
#    'true_category_literal': encoder_category_literal.inverse_transform(y_test),
#    'predicted_category_literal': encoder_category_literal.inverse_transform(y_pred)
#})
#test_results.to_csv('./test_results_catBoost.csv', index=False)
#p2f.reprint("Resultados de prueba guardados en 'test_results_mlp.csv'.")

Archivo original de base, sólo con los rellenos, guardado con éxito.


In [23]:
len(df_complete)

420314